In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
#import cv2 as cv
from osgeo import gdal
import glob
from collections import Counter

In [3]:
from anomalymarinedetection.utils.constants import CLASS_DISTR
from anomalymarinedetection.utils.bands import get_patch_name_from_prediction_name
from anomalymarinedetection.utils.bands import get_tile_name_from_prediction_name, get_patch_name_from_prediction_name
from anomalymarinedetection.utils.assets import labels
from anomalymarinedetection.io.tif_io import TifIO

In [4]:
seed = 0
folder_patches = "../../data/patches/" #"../../data/predicted_unet/"
folder_input_images = "../../data/patches/"
tif_ext = ".tif"

tif_io = TifIO() 

In [5]:
for file_name in os.listdir(folder_patches):
    print(file_name)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../data/patches/'

In [ ]:
i = 0
agg_to_water = False
plot = True
total_pixels_dataset = {}
desired_patch= "S2_20-4-18_30VWH"
for file_name in os.listdir(folder_patches):
    annotaions_counter = len(glob.glob1(folder_patches + file_name,"*_cl.tif"))
    for idx_patch in range(annotaions_counter):
        if file_name == desired_patch:
            index_sub_plot = 0
            tile_name = get_tile_name_from_prediction_name(file_name)
            patch_name = get_patch_name_from_prediction_name(file_name)
            if plot:
                f, axarr = plt.subplots(1, 5, figsize=(15, 5))
                
                
                # Plot rgb image
                tif_path = folder_input_images + tile_name + "/" + patch_name + "_" + str(idx_patch) + tif_ext
                img_rgb_z, coords = tif_io.tif_2_rgb(tif_path)
                axarr[index_sub_plot].set_title("Rgb image")
                axarr[index_sub_plot].imshow(img_rgb)
                index_sub_plot += 1
                # Plot swir image
                tif_path = folder_input_images + tile_name + "/" + patch_name + "_" + str(idx_patch) + tif_ext
                img_rgb = tif_io.tif_2_swir(tif_path)
                axarr[index_sub_plot].set_title("Swir image")
                axarr[index_sub_plot].imshow(img_rgb)
                index_sub_plot += 1
                # Plot fdi image
                #tif_path = folder_input_images + tile_name + "/" + patch_name + "_" + str(idx_patch) + tif_ext
                #img_rgb = tif_io.tif_2_fdi(tif_path)
                #axarr[index_sub_plot].set_title("Fdi image")
                #axarr[index_sub_plot].imshow(img_rgb)
                #index_sub_plot += 1
                # Plot ndvi image
                #tif_path = folder_input_images + tile_name + "/" + patch_name + "_" + str(idx_patch) + tif_ext
                #img_rgb = tif_io.tif_2_ndvi(tif_path)
                #axarr[index_sub_plot].set_title("Ndvi image")
                #axarr[index_sub_plot].imshow(img_rgb)
                #index_sub_plot += 1
                
            
            # Plot semantic segmentation map
            roi_file_cl = folder_input_images + tile_name + "/" + patch_name + "_" + str(idx_patch) + "_cl" + tif_ext        
            ds = gdal.Open(roi_file_cl)
            temp = np.copy(ds.ReadAsArray().astype(np.int64))
            unique, counts = np.unique(temp, return_counts=True)
            
            labels_in_img = [labels[label_idx - 1] if label_idx > 0 else "0" for label_idx in unique]
            pixels_in_img = dict(zip(labels_in_img, counts))
            
            total_pixels_dataset = dict(Counter(total_pixels_dataset) + Counter(pixels_in_img))

            
            if plot:
                axarr[index_sub_plot].set_title("GT")
                axarr[index_sub_plot].imshow(temp)
                index_sub_plot += 1
            
            # Aggregation
            if agg_to_water:
                temp[temp == 15] = 7  # Mixed Water to Marine Water Class
                temp[temp == 14] = 7  # Wakes to Marine Water Class
                temp[temp == 13] = 7  # Cloud Shadows to Marine Water Class
                temp[temp == 12] = 7  # Waves to Marine Water Class

            # Categories from 1 to 0
            temp = np.copy(temp - 1)
            
            
            # Plot prediction
            #path = folder_predictions + file_name
            #img_pred, _ = acquire_data(path)
            #axarr[index_sub_plot].set_title("Prediction")
            #axarr[index_sub_plot].imshow(img_pred)
            
            print("_" * 40 + f"{file_name}"+ "_" * 40)
            print(pixels_in_img) # pixels for each category in the current image
            if plot:
                plt.show()
    
    i += 1

print("_" * 80 + "\nPixels for each category of the looped images:")
print(total_pixels_dataset)

In [ ]:
#import rasterio.mask as mask
#out_img, out_transform = mask(raster=, shapes=coords, crop=True)

In [ ]:
"""
i = 0

for file_name in os.listdir(folder_predictions):
    if i >= 50 and i < 70:
        index_sub_plot = 0
        tile_name = get_tile_name_from_prediction_name(file_name)
        patch_name = get_patch_name_from_prediction_name(file_name)
        
        f, axarr = plt.subplots(1, 5, figsize=(15, 5))
        
        
        # Plot rgb image
        tif_path = folder_input_images + tile_name + "/" + patch_name + "_0" + tif_ext
        img_rgb = tif_io.tif_2_rgb(tif_path)
        axarr[index_sub_plot].set_title("Rgb image")
        axarr[index_sub_plot].imshow(img_rgb)
        index_sub_plot += 1
        # Plot swir image
        tif_path = folder_input_images + tile_name + "/" + patch_name + "_0" + tif_ext
        img_rgb = tif_io.tif_2_swir(tif_path)
        axarr[index_sub_plot].set_title("Swir image")
        axarr[index_sub_plot].imshow(img_rgb)
        index_sub_plot += 1
        # Plot fdi image
        tif_path = folder_input_images + tile_name + "/" + patch_name + "_0" + tif_ext
        img_rgb = tif_io.tif_2_fdi(tif_path)
        axarr[index_sub_plot].set_title("Fdi image")
        axarr[index_sub_plot].imshow(img_rgb)
        index_sub_plot += 1
        # Plot ndvi image
        tif_path = folder_input_images + tile_name + "/" + patch_name + "_0" + tif_ext
        img_rgb = tif_io.tif_2_ndvi(tif_path)
        axarr[index_sub_plot].set_title("Ndvi image")
        axarr[index_sub_plot].imshow(img_rgb)
        index_sub_plot += 1
        # Plot semantic segmentation map
        tif_path = folder_input_images + tile_name + "/" + patch_name + "_0" + "_cl" + tif_ext
        img_pred, _ = acquire_data(tif_path)
        axarr[index_sub_plot].set_title("GT")
        axarr[index_sub_plot].imshow(img_pred)
        index_sub_plot += 1
        # Plot prediction
        #path = folder_predictions + file_name
        #img_pred, _ = acquire_data(path)
        #axarr[index_sub_plot].set_title("Prediction")
        #axarr[index_sub_plot].imshow(img_pred)
        
        print("_" * 40 + f"{file_name}"+ "_" * 40)

        plt.show()
    
    i += 1
"""

In [ ]:
import os
import ast
import sys
import json
import random
import logging
import argparse
import numpy as np
from tqdm import tqdm
from os.path import dirname as up

import torch
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from anomalymarinedetection.semantic_segmentation.dataloader import (
    AnomalyMarineDataset,
    bands_mean,
    bands_std,
    RandomRotationTransform,
    class_distr,
    gen_weights,
)


def seed_all(seed):
    # Pytorch Reproducibility
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def seed_worker(worker_id):
    # DataLoader Workers Reproducibility
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

seed_all(0)
g = torch.Generator()
g.manual_seed(0)

# Transformations

transform_train = transforms.Compose(
    [
        transforms.ToTensor(),
        RandomRotationTransform([-90, 0, 90, 180]),
        transforms.RandomHorizontalFlip(),
    ]
)

transform_test = transforms.Compose([transforms.ToTensor()])

standardization = transforms.Normalize(bands_mean, bands_std)


dataset_train = AnomalyMarineDataset(
    "train",
    transform=transform_train,
    standardization=standardization,
    agg_to_water=True,
    path="/data/anomaly-marine-detection/data"
)


train_loader = DataLoader(
    dataset_train,
    batch_size=1,
    shuffle=True,
    num_workers=1,
    pin_memory=True,
    prefetch_factor=True,
    persistent_workers=True,
    worker_init_fn=seed_worker,
    generator=g,
)

In [ ]:
def s2_2_rgb(s2_img: str) -> np.ndarray:
    img_b = s2_img[1, :, :].reshape(s2_img.shape[-2], s2_img.shape[-1], 1)
    img_g = s2_img[2, :, :].reshape(s2_img.shape[-2], s2_img.shape[-1], 1)
    img_r = s2_img[3, :, :].reshape(s2_img.shape[-2], s2_img.shape[-1], 1)

    img_rgb = np.concatenate((img_r, img_g, img_b), 2)
    print(img_rgb.shape)
    img_rgb = img_rgb / img_rgb.max()
    return img_rgb

In [ ]:
j = 0
n = 2
for img, target in train_loader:
    img = img[0]
    print(torch.min(img), torch.max(img))
    print(img.shape)
    img = np.array(img)
    print(img.shape)
    j += 1
    rgb_img = s2_2_rgb(img)
    print(np.unique(target))
    print(np.min(rgb_img), np.max(rgb_img))
    plt.imshow(rgb_img, vmin=0, vmax=255)
    plt.show()
    plt.imshow(target[0])
    plt.show()
    if j > n:
        break